Import Module

In [1]:
import scPAFA
import scanpy as sc
import json

Use scanpy to read anndata in h5ad format, which contain single-cell gene expression matrix

In [2]:
#read_adata_single_cell
#lupus_dataset
adata = sc.read_h5ad('/data/scPAFA_paper/lupus_file/20231115_lupus_raw_geneover30_cellxgene.h5ad')

In [3]:
#1263676cells*22444genes
adata.shape

(1263676, 20514)

Import pathway collection

In [4]:
#read_pathway_dict
with open('../example_file/pathwaydict_bioplanet.json', 'r') as file:
    pathway_dict = json.load(file)

Quality control and reformat of pathway collection

In [5]:
#pathway_input_qc_by_overlapgene
pathway_input = scPAFA.tl.generate_pathway_input(adata = adata,pathway_dict= pathway_dict,min_overlap_gene=6)

Filtered out 275 pathways
1383 pathways passed QC
The maxRank must >= 1615 (The genes number of the longest pathway)


Calculation of PAS

In [6]:
#step1_generate_rank_matrix
rank_matrix = scPAFA.tl.fast_ucell_rank(adata=adata,maxRank=2000,n_cores_rank=10,rank_batch_size=100000)
#step2_UCell_compute
Ucell_result =  scPAFA.tl.fast_ucell_score(cell_index = list(adata.obs.index),rankmatrix=rank_matrix,maxRank=2000,n_cores_score=10,score_batch_size = 100000,input_dict = pathway_input)

adata.X is csr sparse matrix
Step1 generate rank matrix
13 batches need to rank, with each max 100000 cells
Processing_batch_1


Ranking Chunks: 100%|██████████| 10/10 [02:08<00:00, 12.89s/it]


Processing_batch_2


Ranking Chunks: 100%|██████████| 10/10 [01:55<00:00, 11.54s/it]


Processing_batch_3


...

processing_batch_12


Pathways: 100%|██████████| 1383/1383 [00:52<00:00, 26.21it/s]


processing_batch_13


Pathways: 100%|██████████| 1383/1383 [00:37<00:00, 37.17it/s]


Ucell_done
Outputing_dataframe


Save PAS

In [7]:
#save UCell_result as parquet file
import pandas as pd
Ucell_result.to_parquet('/data/scPAFA_paper/lupus_file/Ucell_lupus_bioplanet_example.parquet')